# Located names

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores4.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 5.15 s, sys: 201 ms, total: 5.35 s
Wall time: 5.35 s


In [3]:
autores_df.shape

(3054360, 5)

In [4]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,2010,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,2010,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,2010,Spain,Sonia Alins,DESCONOCIDO


# Autores con país conocido

In [5]:
autores_pais_conocido_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] != 'DESCONOCIDO')
]

In [6]:
autores_pais_conocido_df.shape

(811460, 5)

In [7]:
autores_pais_conocido_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
10,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany
11,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany
12,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany
21,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom
22,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom


In [8]:
autores_pais_conocido_df['AUTOR'].nunique()

13366

## País autor

In [9]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            {{
                {{
                    ?author wdt:P106 wd:Q36180 .
                }}
                UNION
                {{ 
                    ?author wdt:P106 ?prof .
                    ?prof wdt:P279 wd:Q36180 .
                }}
            }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [10]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            {{
                {{
                    ?author rdf:type dbo:Writer .
                }}
                UNION
                {{
                    ?author rdf:type ?prof .
                    ?prof rdfs:subClassOf dbo:Writer .
                }}
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [11]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [12]:
def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['wdt', 'dbp']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [13]:
%%time

autores_paises_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_paises_dict[x['AUTOR']] = x['PAIS_AUTOR']

CPU times: user 19.2 s, sys: 16.1 ms, total: 19.2 s
Wall time: 19.2 s


In [14]:
def combinaciones(ls):
    r = []
    for x in ls:
        aux = list(ls)
        aux.remove(x)
        r.append(aux)
    return r

In [15]:
%%time

nombre_localizado_dict = {}

for autor in autores_pais_conocido_df['AUTOR'].unique():
    pais = get_pais_autor(autor)
    if pais != 'DESCONOCIDO':
        nombre_localizado_dict[autor] = autor
    else:
        nombres = autor.split()
        if len(nombres) > 2:
            for c in combinaciones(nombres):
                aux = ' '.join(c)
                pais = get_pais_autor(aux)
                if pais != 'DESCONOCIDO':
                    nombre_localizado_dict[autor] = aux
                    break

MáS DE UN PAíS PARA Ludmila Zeman : {'Czech Republic', 'Czechoslovakia'}
MáS DE UN PAíS PARA David Shannon : {'United Kingdom', 'United States of America'}
MáS DE UN PAíS PARA Mario Ramos : {'Spain', 'Belgium'}
MáS DE UN PAíS PARA Mario Benedetti : {'Uruguay', 'Italy'}
MáS DE UN PAíS PARA Antonin Artaud : {'Spain', 'Monaco', 'Italy', 'France'}
MáS DE UN PAíS PARA Yotam Ottolenghi : {'Mandatory Palestine', 'Jordan', 'Palestine', 'Ottoman Empire', 'Israel'}
MáS DE UN PAíS PARA Richard Wagner : {'Romania', 'Germany'}
MáS DE UN PAíS PARA Robert Taylor : {'United Kingdom', 'United States of America'}
MáS DE UN PAíS PARA James Morrison : {'Australia', 'United States of America'}
MáS DE UN PAíS PARA James Martin : {'United Kingdom', 'United States of America'}
MáS DE UN PAíS PARA David Grossman : {'Mandatory Palestine', 'Jordan', 'Palestine', 'Ottoman Empire', 'Israel'}
MáS DE UN PAíS PARA John Rogers : {'United Kingdom', 'United States of America'}
MáS DE UN PAíS PARA Joseph Campbell : {'Uni

In [16]:
%%time

def process_nombre_localizado(x):
    return nombre_localizado_dict.get(x['AUTOR'], 'DESCONOCIDO')

autores_df['NOMBRE_LOCALIZADO'] = autores_df.apply(process_nombre_localizado, axis=1)

CPU times: user 1min 19s, sys: 62.3 ms, total: 1min 19s
Wall time: 1min 19s


# Saving results

In [17]:
autores_df.to_csv('autores5.csv', index=False)